<a href="https://colab.research.google.com/github/MSebastian3/Evaluacion/blob/main/Actividad_5_(Evaluaci%C3%B3n).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [977]:
#Instalamos librería para análisis de variables categóricas
#%pip install funpymodeling

In [978]:
#Importamos librerías requeridas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#from funpymodeling.exploratory import freq_tbl 

In [979]:
#Cargar archivo csv desde equipo
#from google.colab import files
#files.upload()

# ***California_EUA.csv***

## **ETAPA 1**

In [980]:
dfeua = pd.read_csv('California_EUA.csv', encoding = 'unicode_escape')
dfeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6936 entries, 0 to 6935
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ï»¿id                                         6936 non-null   float64
 1   listing_url                                   6936 non-null   object 
 2   scrape_id                                     6936 non-null   float64
 3   last_scraped                                  6936 non-null   object 
 4   source                                        6936 non-null   object 
 5   name                                          6936 non-null   object 
 6   description                                   6889 non-null   object 
 7   neighborhood_overview                         4850 non-null   object 
 8   picture_url                                   6936 non-null   object 
 9   host_id                                       6936 non-null   i

In [981]:
#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=dfeua.isnull().sum().sum()
valores_nulos

48877

### Filtros

In [982]:
#Crear filtro numerico y no numerico
filtroeuanumerico = dfeua.select_dtypes(include=['float64', 'int64'])
#filtroeuanumerico.info()

In [983]:
filtroeua = dfeua.select_dtypes('object')
#filtroeuanonumerico.info()

### Nulos

In [984]:
#sustituir los nulos por el promedio de la columna
data1 = filtroeuanumerico.copy()
filtroeuanumerico = data1.fillna(data1.mean())


#sustituir los nulos por 'sin registro'
filtroeua = filtroeua.fillna('sin registro')


In [985]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

20808

In [986]:
#Checar si hay valores nulos por columna    NO NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeua.isnull().sum().sum()
valores_nulos

0

In [987]:
#NUMERICO
#Se eliminaron las columnas 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'
#ya que son numericas pero no tienen ningun registro
filtroeuanumerico = filtroeuanumerico.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'], axis=1)

#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

0

### Outliers **MÉTODO DE IQR (RANGO INTER QUARTILICO)**

In [988]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtroeuanumerico

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
#print("Limite superior permitido", Limite_Superior_iqr)
#print("Limite inferior permitido", Limite_Inferior_iqr)

In [989]:
#Encontramos Ouliers
outliers_iqr= filtroeuanumerico[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
#outliers_iqr.head()

In [990]:
#Obtenemos datos limpios
data_clean_iqr1= filtroeuanumerico[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
#data_clean_iqr1.head()

In [991]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data_clean_iqr1.isnull().sum().sum()
valores_nulos

17526

In [992]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data5_iqr=filtroeuanumerico.copy()
data5_iqr=data5_iqr.fillna(round(filtroeuanumerico.mean(),1))
#data5_iqr

In [993]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data5_iqr.isnull().sum().sum()
valores_nulos

0

### Concatenar

In [994]:
concatenadoeua = pd.concat([data5_iqr, filtroeua], axis=1)
concatenadoeua.head()

,ï»¿id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,958.0,2.020000e+13,1169,1,1,37.77028,-122.43317,3,1.0,2.0,...,Entire home/apt,1 bath,"[""Oven"", ""Self check-in"", ""Cooking basics"", ""L...",$202.00,t,06/03/2023,23/07/2009,19/02/2023,City Registration Pending,f
1,5858.0,2.020000e+13,8904,2,2,37.74474,-122.42089,4,2.0,2.0,...,Entire home/apt,1 bath,"[""Essentials"", ""Carbon monoxide alarm"", ""Hange...",$235.00,t,06/03/2023,03/05/2009,06/08/2017,sin registro,f
2,8142.0,2.020000e+13,21994,13,14,37.76555,-122.45213,1,1.0,1.0,...,Private room,4 shared baths,"[""Security cameras on property"", ""Host greets ...",$56.00,t,06/03/2023,08/09/2014,27/10/2022,sin registro,f
3,8339.0,2.020000e+13,24215,2,6,37.77564,-122.43642,4,2.0,2.0,...,Entire home/apt,1.5 baths,"[""Oven"", ""Cooking basics"", ""TV"", ""Smoke alarm""...",$575.00,t,06/03/2023,25/09/2009,28/06/2019,STR-0000264,f
4,8739.0,2.020000e+13,7149,2,2,37.76030,-122.42197,3,1.0,1.0,...,Private room,1 private bath,"[""Paid parking garage off premises"", ""Luggage ...",$110.00,t,06/03/2023,01/08/2009,25/02/2023,STR-0000028,f


### Guardar a CSV

In [995]:
#Convertir archivo filtrado a CSV
concatenadoeua.to_csv("EUA_limpio.csv")

In [996]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("EUA_limpio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **ETAPA 2**

In [997]:
#filtroa
#se cambiaron los valores "sin registro" por "5%" para poder identificarlos
concatenadoeua["host_acceptance_rate"] = concatenadoeua["host_acceptance_rate"].replace("sin registro", "5%")
#se elimino el "%"
concatenadoeua['host_acceptance_rate'] = pd.to_numeric(concatenadoeua['host_acceptance_rate'].str.replace('%', ''))



In [998]:
#filtroe
#cambiar Half-bath, Shared half-bath, Private half-bath, sin registro
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.replace('Half-bath', '0.5').str.replace('Shared half-bath', '0.5').str.replace('Private half-bath', '0.5').str.replace('sin registro', '0')

#se extraen los numeros y se guardan como string en la misma columna
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.extract('(\d+\.?\d*)', expand=False)

#cambiar columna a float
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].astype(float)

In [999]:
#filtrof
#se elimino el "$" y ","
#se cambia la columna "price" a float para poder compararla
concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


<ipython-input-999-af53cb0f8d5c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


In [1000]:
filtroa = concatenadoeua[concatenadoeua["host_acceptance_rate"] > 50]
filtrob = concatenadoeua[concatenadoeua["host_is_superhost"] == "t"]
filtroc = concatenadoeua[concatenadoeua["host_identity_verified"] == "f"]
filtrod = concatenadoeua[concatenadoeua["property_type"].isin(["Private room", "Hotel room"])]
filtroe = concatenadoeua[concatenadoeua["bathrooms_text"] > 1]
filtrof = concatenadoeua.loc[(concatenadoeua['price'] > 10000) & (concatenadoeua['room_type'] == 'Entire home/apt')]
filtrog = concatenadoeua[concatenadoeua["review_scores_cleanliness"] > 4.5]
filtroh = concatenadoeua[concatenadoeua["review_scores_value"] > 4.9]
filtroi = concatenadoeua[concatenadoeua["availability_365"] < 100]
filtroj = concatenadoeua[concatenadoeua["host_response_time"] == "within an hour"]

### Guardar a CSV

In [1001]:
#Convertir archivo filtrado a CSV
filtroa.to_csv("filtroa_eua.csv")
filtrob.to_csv("filtrob_eua.csv")
filtroc.to_csv("filtroc_eua.csv")
filtrod.to_csv("filtrod_eua.csv")
filtroe.to_csv("filtroe_eua.csv")
filtrof.to_csv("filtrof_eua.csv")
filtrog.to_csv("filtrog_eua.csv")
filtroh.to_csv("filtroh_eua.csv")
filtroi.to_csv("filtroi_eua.csv")
filtroj.to_csv("filtroj_eua.csv")

In [1002]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("filtroa_eua.csv")
files.download("filtrob_eua.csv")
files.download("filtroc_eua.csv")
files.download("filtrod_eua.csv")
files.download("filtroe_eua.csv")
files.download("filtrof_eua.csv")
files.download("filtrog_eua.csv")
files.download("filtroh_eua.csv")
files.download("filtroi_eua.csv")
files.download("filtroj_eua.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ***Girona_España.csv***

## **ETAPA 1**

In [1003]:
dfeua = pd.read_csv('Girona_España.csv', encoding = 'unicode_escape')
dfeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19655 entries, 0 to 19654
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            19655 non-null  float64
 1   listing_url                                   19655 non-null  object 
 2   scrape_id                                     19655 non-null  float64
 3   last_scraped                                  19655 non-null  object 
 4   source                                        19655 non-null  object 
 5   name                                          19655 non-null  object 
 6   description                                   19632 non-null  object 
 7   neighborhood_overview                         8342 non-null   object 
 8   picture_url                                   19655 non-null  object 
 9   host_id                                       19655 non-null 

In [1004]:
#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=dfeua.isnull().sum().sum()
valores_nulos

174630

### Filtros

In [1005]:
#Crear filtro numerico y no numerico
filtroeuanumerico = dfeua.select_dtypes(include=['float64', 'int64'])
#filtroeuanumerico.info()

In [1006]:
filtroeua = dfeua.select_dtypes('object')
#filtroeuanonumerico.info()

### Nulos

In [1007]:
#sustituir los nulos por el promedio de la columna
data1 = filtroeuanumerico.copy()
filtroeuanumerico = data1.fillna(data1.mean())


#sustituir los nulos por 'sin registro'
filtroeua = filtroeua.fillna('sin registro')


In [1008]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

58965

In [1009]:
#Checar si hay valores nulos por columna    NO NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeua.isnull().sum().sum()
valores_nulos

0

In [1010]:
#NUMERICO
#Se eliminaron las columnas 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'
#ya que son numericas pero no tienen ningun registro
filtroeuanumerico = filtroeuanumerico.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'], axis=1)

#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

0

### Outliers **MÉTODO DE IQR (RANGO INTER QUARTILICO)**

In [1011]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtroeuanumerico

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
#print("Limite superior permitido", Limite_Superior_iqr)
#print("Limite inferior permitido", Limite_Inferior_iqr)

In [1012]:
#Encontramos Ouliers
outliers_iqr= filtroeuanumerico[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
#outliers_iqr.head()

In [1013]:
#Obtenemos datos limpios
data_clean_iqr1= filtroeuanumerico[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
#data_clean_iqr1.head()

In [1014]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data_clean_iqr1.isnull().sum().sum()
valores_nulos

37109

In [1015]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data5_iqr=filtroeuanumerico.copy()
data5_iqr=data5_iqr.fillna(round(filtroeuanumerico.mean(),1))
#data5_iqr

In [1016]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data5_iqr.isnull().sum().sum()
valores_nulos

0

### Concatenar

In [1017]:
concatenadoeua = pd.concat([data5_iqr, filtroeua], axis=1)
concatenadoeua.head()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,14551.0,2.023030e+13,57237,1.0,5.0,41.99650,2.83646,2,1.0,1.0,...,Private room,1 private bath,"[""Shared pool"", ""Refrigerator"", ""Long term sta...",$42.00,t,31/03/2023,08/10/2010,27/02/2023,HUTG-049542,f
1,1205757.0,2.023030e+13,6586355,1.0,1.0,41.79304,2.77412,2,1.0,1.0,...,Private room,1.5 shared baths,"[""Free parking on premises"", ""Refrigerator"", ""...",$47.00,t,31/03/2023,01/08/2013,26/06/2022,sin registro,f
2,1212820.0,2.023030e+13,6620645,2.0,3.0,42.11284,3.16670,4,1.0,2.0,...,Entire home/apt,1 bath,"[""Air conditioning"", ""Free parking on premises...",$74.00,t,01/04/2023,10/06/2013,30/12/2022,sin registro,f
3,751638.0,2.023030e+13,3680160,6.0,6.0,42.36958,2.98030,2,1.0,2.0,...,Private room,1 private bath,"[""Hair dryer"", ""Luggage dropoff allowed"", ""Fre...",$45.00,t,31/03/2023,23/08/2013,29/08/2022,PG000794,t
4,41342.0,2.023030e+13,180072,2.0,3.0,42.34655,3.20463,4,1.0,1.0,...,Entire home/apt,1 bath,"[""Free parking on premises"", ""Refrigerator"", ""...",$120.00,t,01/04/2023,14/07/2013,28/08/2021,HUTG-003688,f


### Guardar a CSV

In [1018]:
#Convertir archivo filtrado a CSV
concatenadoeua.to_csv("ESP_limpio.csv")

In [1019]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("ESP_limpio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **ETAPA 2**

In [1020]:
#filtroa
#se cambiaron los valores "sin registro" por "5%" para poder identificarlos
concatenadoeua["host_acceptance_rate"] = concatenadoeua["host_acceptance_rate"].replace("sin registro", "5%")
#se elimino el "%"
concatenadoeua['host_acceptance_rate'] = pd.to_numeric(concatenadoeua['host_acceptance_rate'].str.replace('%', ''))



In [1021]:
#filtroe
#cambiar Half-bath, Shared half-bath, Private half-bath, sin registro
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.replace('Half-bath', '0.5').str.replace('Shared half-bath', '0.5').str.replace('Private half-bath', '0.5').str.replace('sin registro', '0')

#se extraen los numeros y se guardan como string en la misma columna
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.extract('(\d+\.?\d*)', expand=False)

#cambiar columna a float
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].astype(float)

In [1022]:
#filtrof
#se elimino el "$" y ","
#se cambia la columna "price" a float para poder compararla
concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


<ipython-input-1022-af53cb0f8d5c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


In [1023]:
filtroa = concatenadoeua[concatenadoeua["host_acceptance_rate"] > 50]
filtrob = concatenadoeua[concatenadoeua["host_is_superhost"] == "t"]
filtroc = concatenadoeua[concatenadoeua["host_identity_verified"] == "f"]
filtrod = concatenadoeua[concatenadoeua["property_type"].isin(["Private room", "Hotel room"])]
filtroe = concatenadoeua[concatenadoeua["bathrooms_text"] > 1]
filtrof = concatenadoeua.loc[(concatenadoeua['price'] > 10000) & (concatenadoeua['room_type'] == 'Entire home/apt')]
filtrog = concatenadoeua[concatenadoeua["review_scores_cleanliness"] > 4.5]
filtroh = concatenadoeua[concatenadoeua["review_scores_value"] > 4.9]
filtroi = concatenadoeua[concatenadoeua["availability_365"] < 100]
filtroj = concatenadoeua[concatenadoeua["host_response_time"] == "within an hour"]

### Guardar a CSV

In [1024]:
#Convertir archivo filtrado a CSV
filtroa.to_csv("filtroa_esp.csv")
filtrob.to_csv("filtrob_esp.csv")
filtroc.to_csv("filtroc_esp.csv")
filtrod.to_csv("filtrod_esp.csv")
filtroe.to_csv("filtroe_esp.csv")
filtrof.to_csv("filtrof_esp.csv")
filtrog.to_csv("filtrog_esp.csv")
filtroh.to_csv("filtroh_esp.csv")
filtroi.to_csv("filtroi_esp.csv")
filtroj.to_csv("filtroj_esp.csv")

In [1025]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("filtroa_esp.csv")
files.download("filtrob_esp.csv")
files.download("filtroc_esp.csv")
files.download("filtrod_esp.csv")
files.download("filtroe_esp.csv")
files.download("filtrof_esp.csv")
files.download("filtrog_esp.csv")
files.download("filtroh_esp.csv")
files.download("filtroi_esp.csv")
files.download("filtroj_esp.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ***DF_Mexico.csv***

## **ETAPA 1**

In [1026]:
dfeua = pd.read_csv('DF_Mexico.csv')#, encoding = 'unicode_escape')
dfeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24224 entries, 0 to 24223
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            24224 non-null  float64
 1   listing_url                                   24224 non-null  object 
 2   scrape_id                                     24224 non-null  float64
 3   last_scraped                                  24224 non-null  object 
 4   source                                        24224 non-null  object 
 5   name                                          24223 non-null  object 
 6   description                                   23296 non-null  object 
 7   neighborhood_overview                         15145 non-null  object 
 8   picture_url                                   24224 non-null  object 
 9   host_id                                       24224 non-null 

In [1027]:
#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=dfeua.isnull().sum().sum()
valores_nulos

189609

### Filtros

In [1028]:
#Crear filtro numerico y no numerico
filtroeuanumerico = dfeua.select_dtypes(include=['float64', 'int64'])
#filtroeuanumerico.info()

In [1029]:
filtroeua = dfeua.select_dtypes('object')
#filtroeuanonumerico.info()

### Nulos

In [1030]:
#sustituir los nulos por el promedio de la columna
data1 = filtroeuanumerico.copy()
filtroeuanumerico = data1.fillna(data1.mean())


#sustituir los nulos por 'sin registro'
filtroeua = filtroeua.fillna('sin registro')


In [1031]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

96896

In [1032]:
#Checar si hay valores nulos por columna    NO NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeua.isnull().sum().sum()
valores_nulos

0

In [1033]:
#NUMERICO
#Se eliminaron las columnas 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'license'
#ya que son numericas pero no tienen ningun registro
filtroeuanumerico = filtroeuanumerico.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated','license'], axis=1)

#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

0

### Outliers **MÉTODO DE IQR (RANGO INTER QUARTILICO)**

In [1034]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtroeuanumerico

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
#print("Limite superior permitido", Limite_Superior_iqr)
#print("Limite inferior permitido", Limite_Inferior_iqr)

In [1035]:
#Encontramos Ouliers
outliers_iqr= filtroeuanumerico[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
#outliers_iqr.head()

In [1036]:
#Obtenemos datos limpios
data_clean_iqr1= filtroeuanumerico[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
#data_clean_iqr1.head()

In [1037]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data_clean_iqr1.isnull().sum().sum()
valores_nulos

54132

In [1038]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data5_iqr=filtroeuanumerico.copy()
data5_iqr=data5_iqr.fillna(round(filtroeuanumerico.mean(),1))
#data5_iqr

In [1039]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data5_iqr.isnull().sum().sum()
valores_nulos

0

### Concatenar

In [1040]:
concatenadoeua = pd.concat([data5_iqr, filtroeua], axis=1)
concatenadoeua.head()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,property_type,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,instant_bookable
0,2056638.0,2.023030e+13,10531228,3,7,19.35353,-99.16299,3,1.0,1.0,...,Entire loft,Entire home/apt,1.5 baths,"[""Wifi"", ""Hangers"", ""Fire extinguisher"", ""Sham...","$1,434.00",t,30/03/23,03/01/14,24/06/18,f
1,2072354.0,2.023030e+13,16840050,3,3,19.35358,-99.16948,2,1.0,1.0,...,Entire guest suite,Entire home/apt,1 bath,"[""Free street parking"", ""Cleaning products"", ""...",$830.00,t,30/03/23,01/01/14,04/11/22,f
2,696037.0,2.023030e+13,3531879,4,4,19.44180,-99.18402,6,3.0,3.0,...,Entire rental unit,Entire home/apt,2 baths,"[""Wifi \u2013 38 Mbps"", ""Central air condition...","$1,469.00",t,29/03/23,31/10/12,21/12/22,f
3,35797.0,2.023030e+13,153786,1,1,19.38283,-99.27178,2,1.0,1.0,...,Entire villa,Entire home/apt,1 bath,"[""Wifi"", ""Washer"", ""BBQ grill"", ""Indoor firepl...","$3,658.00",t,29/03/23,sin registro,sin registro,f
4,44616.0,2.023030e+13,196253,16,16,19.41162,-99.17794,14,5.0,8.0,...,Entire home,Entire home/apt,5.5 baths,"[""Free street parking"", ""Security cameras on p...","$18,000.00",t,30/03/23,09/11/11,26/03/23,f


### Guardar a CSV

In [1041]:
#Convertir archivo filtrado a CSV
concatenadoeua.to_csv("MEX_limpio.csv")

In [1042]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("MEX_limpio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **ETAPA 2**

In [1043]:
#filtroa
#se cambiaron los valores "sin registro" por "5%" para poder identificarlos
concatenadoeua["host_acceptance_rate"] = concatenadoeua["host_acceptance_rate"].replace("sin registro", "5%")
#se elimino el "%"
concatenadoeua['host_acceptance_rate'] = pd.to_numeric(concatenadoeua['host_acceptance_rate'].str.replace('%', ''))



In [1044]:
#filtroe
#cambiar Half-bath, Shared half-bath, Private half-bath, sin registro
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.replace('Half-bath', '0.5').str.replace('Shared half-bath', '0.5').str.replace('Private half-bath', '0.5').str.replace('sin registro', '0')

#se extraen los numeros y se guardan como string en la misma columna
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.extract('(\d+\.?\d*)', expand=False)

#cambiar columna a float
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].astype(float)

In [1045]:
#filtrof
#se elimino el "$" y ","
#se cambia la columna "price" a float para poder compararla
concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


<ipython-input-1045-af53cb0f8d5c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


In [1046]:
filtroa = concatenadoeua[concatenadoeua["host_acceptance_rate"] > 50]
filtrob = concatenadoeua[concatenadoeua["host_is_superhost"] == "t"]
filtroc = concatenadoeua[concatenadoeua["host_identity_verified"] == "f"]
filtrod = concatenadoeua[concatenadoeua["property_type"].isin(["Private room", "Hotel room"])]
filtroe = concatenadoeua[concatenadoeua["bathrooms_text"] > 1]
filtrof = concatenadoeua.loc[(concatenadoeua['price'] > 10000) & (concatenadoeua['room_type'] == 'Entire home/apt')]
filtrog = concatenadoeua[concatenadoeua["review_scores_cleanliness"] > 4.5]
filtroh = concatenadoeua[concatenadoeua["review_scores_value"] > 4.9]
filtroi = concatenadoeua[concatenadoeua["availability_365"] < 100]
filtroj = concatenadoeua[concatenadoeua["host_response_time"] == "within an hour"]

### Guardar a CSV

In [1047]:
#Convertir archivo filtrado a CSV
filtroa.to_csv("filtroa_mex.csv")
filtrob.to_csv("filtrob_mex.csv")
filtroc.to_csv("filtroc_mex.csv")
filtrod.to_csv("filtrod_mex.csv")
filtroe.to_csv("filtroe_mex.csv")
filtrof.to_csv("filtrof_mex.csv")
filtrog.to_csv("filtrog_mex.csv")
filtroh.to_csv("filtroh_mex.csv")
filtroi.to_csv("filtroi_mex.csv")
filtroj.to_csv("filtroj_mex.csv")

In [1048]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("filtroa_mex.csv")
files.download("filtrob_mex.csv")
files.download("filtroc_mex.csv")
files.download("filtrod_mex.csv")
files.download("filtroe_mex.csv")
files.download("filtrof_mex.csv")
files.download("filtrog_mex.csv")
files.download("filtroh_mex.csv")
files.download("filtroi_mex.csv")
files.download("filtroj_mex.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>